The purpose of this notebook is to demonstrate how to do a single refinement with the pySULI library modules.



# General imports

In [1]:
# We need to import some Python libraries here first.

import os
import sys
import shutil
import scipy
import time
import glob

import numpy as np
import xarray as xr


from IPython.display import clear_output

In [2]:
# We need import pyFAI,PyQt5,ipympl packages (if not available pip-install)


try:
    import ipympl
except:
    !! pip install ipympl

# if running on google colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

In [3]:
try:
    %matplotlib widget
except:
    %matplotlib inline

# importing matplotlib for plots.
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams['figure.constrained_layout.use'] = True

# Setting up the example data path

In [4]:
# We assume this notebook and its data is on Google Drive so that we can use Google Colab.
# Otherwise, replace example_data_path with the path the notebook and its data folder are stored on

try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/SULI-2024/repositories/pySULI/notebooks/')
    from google.colab import output
    output.enable_custom_widget_manager()

except:
    pass

In [5]:
# Assuming notebook and data folder are here.

here = os.getcwd()
example_data_path = 'example_data/xrd_refinement_with_gsas2/'

os.chdir('%s/%s'%(here,example_data_path))
os.listdir()

['gsas.instprm',
 'Ceo2.cif',
 'gsas2_scratch',
 'Pt.cif',
 'Zr.cif',
 'Pt.gpx',
 'LaB6.nc',
 'LaB6.cif',
 'ZrH.nc',
 'Zr-H1.66.cif',
 'Background.nc',
 'ZrH.gpx',
 'Pt.nc',
 'LaB6.gpx']

In [6]:
ls

Background.nc  LaB6.gpx       Pt.gpx         Zr.cif         gsas.instprm
Ceo2.cif       LaB6.nc        Pt.nc          ZrH.gpx        gsas2_scratch/
LaB6.cif       Pt.cif         Zr-H1.66.cif   ZrH.nc


# Setting up GSAS-II package and custom refinement functions

In [7]:
# assuming GSAS-II is installed on the machine on conda
# as explained here: https://advancedphotonsource.github.io/GSAS-II-tutorials/install.html
# as a result of "conda create -n GSASII briantoby::gsas2pkg  -c conda-forge", GSAS-II
# will be installed on a eng location. Then we add it to path as below.

import sys

user = "kmorell"

match user:
    case "mtopsakal":
        sys.path += ['/home/mt/software/miniforge3/envs/GSASII/GSAS-II/GSASII']
    case "pmeshkov":
        sys.path += ['/opt/anaconda3/envs/GSASII/GSAS-II/GSASII']
    case "kmorell"
        sys.path += ['/Users/kevinmorell/g2full/GSAS-II/GSASII']

# we then import GSASIIscriptable
import GSASIIscriptable as G2sc
import pybaselines # this comes with gsas2_package

from IPython.display import clear_output

# clear_output()

GSAS-II binary directory: /opt/anaconda3/envs/GSASII/GSAS-II/GSASII-bin/mac_64_p3.11_n1.26
7 values read from config file /opt/anaconda3/envs/GSASII/GSAS-II/GSASII/config.py
GSASIIplot: wx not imported
ImportError for wx/mpl in GSASIIctrlGUI: ignore if docs build


## GSAS functions in pySULI

All of the GSAS-II functions defined in notebook 005 are contained within the pySULI modules GSAS2_functions and functions_lib_version


# Refinements

In [13]:
os.makedirs('gsas2_scratch',exist_ok=True)

In [15]:
#Defining the background
with xr.open_dataset('Background.nc') as ds:
    da_input_bkg = ds.i2d.mean(dim='azimuthal')

In [ ]:
#Creating the refiner
first_refiner = rf.Refiner(
    nc_path = 'LaB6.nc',
    phases = [{'cif_abs_path':'LaB6.cif','phase_name':'LaB6','scale':1},],
    gsas2_scratch = None,
    instrument_parameters_file = 'gsas2_scratch/gsas.instprm',
    q_range = None,
    GSASII_path = '/Users/kevinmorell/g2full/GSAS-II/GSASII',
    da_input_bkg = None 
#NOTE: Refiner can create its own background if set to None. Try both to see changes in GoF and Rwp
)